In [1]:
from transformers import GPT2Config, GPT2Model, GPT2Tokenizer, GPT2LMHeadModel

/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: 

In [2]:
import os

out = 'gpt2-117M-bahasa-cased'
os.makedirs(out, exist_ok=True)

In [3]:
tokenizer = GPT2Tokenizer('117m-bahasa/bahasa-vocab.json', 
                            '117m-bahasa/bahasa-merges.txt', do_lower_case = False)
tokenizer.save_pretrained('gpt2-117M-bahasa-cased')

('gpt2-117M-bahasa-cased/vocab.json',
 'gpt2-117M-bahasa-cased/merges.txt',
 'gpt2-117M-bahasa-cased/special_tokens_map.json',
 'gpt2-117M-bahasa-cased/added_tokens.json')

In [4]:
import re
import tensorflow as tf

In [5]:
tf_path = os.path.abspath('117m-bahasa-v3/model.ckpt-20000')
init_vars = tf.train.list_variables(tf_path)
names = []
arrays = []
for name, shape in init_vars:
    print("Loading TF weight {} with shape {}".format(name, shape))
    array = tf.train.load_variable(tf_path, name)
    names.append(name)
    arrays.append(array.squeeze())

Loading TF weight global_step with shape []
Loading TF weight model/h0/attn/c_attn/b with shape [2304]
Loading TF weight model/h0/attn/c_attn/b/adam_m with shape [2304]
Loading TF weight model/h0/attn/c_attn/b/adam_v with shape [2304]
Loading TF weight model/h0/attn/c_attn/w with shape [1, 768, 2304]
Loading TF weight model/h0/attn/c_attn/w/adam_m with shape [1, 768, 2304]
Loading TF weight model/h0/attn/c_attn/w/adam_v with shape [1, 768, 2304]
Loading TF weight model/h0/attn/c_proj/b with shape [768]
Loading TF weight model/h0/attn/c_proj/b/adam_m with shape [768]
Loading TF weight model/h0/attn/c_proj/b/adam_v with shape [768]
Loading TF weight model/h0/attn/c_proj/w with shape [1, 768, 768]
Loading TF weight model/h0/attn/c_proj/w/adam_m with shape [1, 768, 768]
Loading TF weight model/h0/attn/c_proj/w/adam_v with shape [1, 768, 768]
Loading TF weight model/h0/ln_1/b with shape [768]
Loading TF weight model/h0/ln_1/b/adam_m with shape [768]
Loading TF weight model/h0/ln_1/b/adam_v 

In [6]:
config = GPT2Config.from_json_file('small-hparams.json')
model = GPT2Model(config)

In [7]:
import torch

for name, array in zip(names, arrays):
    name = name[6:]  # skip "model/"
    name = name.split("/")
    pointer = model
    passed = True
    for m_name in name:
        
        if m_name in ['_step', 'adam_m', 'adam_v']:
            passed = False
            continue
        if re.fullmatch(r"[A-Za-z]+\d+", m_name):
            scope_names = re.split(r"(\d+)", m_name)
        else:
            scope_names = [m_name]
        
        if scope_names[0] == "w" or scope_names[0] == "g":
            pointer = getattr(pointer, "weight")
        elif scope_names[0] == "b":
            pointer = getattr(pointer, "bias")
        elif scope_names[0] == "wpe" or scope_names[0] == "wte":
            pointer = getattr(pointer, scope_names[0])
            pointer = getattr(pointer, "weight")
        else:
            pointer = getattr(pointer, scope_names[0])
        if len(scope_names) >= 2:
            num = int(scope_names[1])
            pointer = pointer[num]
    if passed:
        try:
            assert pointer.shape == array.shape
        except AssertionError as e:
            e.args += (pointer.shape, array.shape)
            raise
        print("Initialize PyTorch weight {}".format(name))
        pointer.data = torch.from_numpy(array)

Initialize PyTorch weight ['h0', 'attn', 'c_attn', 'b']
Initialize PyTorch weight ['h0', 'attn', 'c_attn', 'w']
Initialize PyTorch weight ['h0', 'attn', 'c_proj', 'b']
Initialize PyTorch weight ['h0', 'attn', 'c_proj', 'w']
Initialize PyTorch weight ['h0', 'ln_1', 'b']
Initialize PyTorch weight ['h0', 'ln_1', 'g']
Initialize PyTorch weight ['h0', 'ln_2', 'b']
Initialize PyTorch weight ['h0', 'ln_2', 'g']
Initialize PyTorch weight ['h0', 'mlp', 'c_fc', 'b']
Initialize PyTorch weight ['h0', 'mlp', 'c_fc', 'w']
Initialize PyTorch weight ['h0', 'mlp', 'c_proj', 'b']
Initialize PyTorch weight ['h0', 'mlp', 'c_proj', 'w']
Initialize PyTorch weight ['h1', 'attn', 'c_attn', 'b']
Initialize PyTorch weight ['h1', 'attn', 'c_attn', 'w']
Initialize PyTorch weight ['h1', 'attn', 'c_proj', 'b']
Initialize PyTorch weight ['h1', 'attn', 'c_proj', 'w']
Initialize PyTorch weight ['h1', 'ln_1', 'b']
Initialize PyTorch weight ['h1', 'ln_1', 'g']
Initialize PyTorch weight ['h1', 'ln_2', 'b']
Initialize PyT

In [8]:
from transformers import CONFIG_NAME, WEIGHTS_NAME
CONFIG_NAME, WEIGHTS_NAME

('config.json', 'pytorch_model.bin')

In [9]:
pytorch_dump_folder_path = out
pytorch_weights_dump_path = pytorch_dump_folder_path + "/" + WEIGHTS_NAME
pytorch_config_dump_path = pytorch_dump_folder_path + "/" + CONFIG_NAME

torch.save(model.state_dict(), pytorch_weights_dump_path)

with open(pytorch_config_dump_path, "w", encoding="utf-8") as f:
    f.write(config.to_json_string())

In [10]:
tokenizer = GPT2Tokenizer.from_pretrained('./gpt2-117M-bahasa-cased')

In [11]:
config = GPT2Config.from_json_file('small-hparams.json')
model = GPT2Model.from_pretrained('./gpt2-117M-bahasa-cased/pytorch_model.bin', config = config)

In [12]:
model.save_pretrained('gpt2-117M-bahasa-cased')

In [13]:
model = GPT2LMHeadModel.from_pretrained('./gpt2-117M-bahasa-cased', pad_token_id=tokenizer.eos_token_id)

In [14]:
input_ids = tokenizer.encode('penat bak hang, macam ni aku takmau kerja dah', return_tensors='pt')
input_ids

tensor([[ 2809,   266,  1681, 12167,    15,  1673,   656,   476,   558, 48854,
          1470,   826]])

In [15]:
sample_outputs = model.generate(
    input_ids,
    do_sample=True, 
    max_length=50, 
    top_k=50, 
    top_p=0.95, 
    num_return_sequences=3
)

print("Output:\n" + 100 * '-')
for i, sample_output in enumerate(sample_outputs):
    print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

Output:
----------------------------------------------------------------------------------------------------
0: penat bak hang, macam ni aku takmau kerja dah jadi aku pernah beritahu orang.
Ini bukan aku rasa cam nak ajak teman kan ni.
Tengok ni aku dah ada adik-adik & anak yang tinggal dan kerja2 yang kat sekolah.
1: penat bak hang, macam ni aku takmau kerja dah.
Takleh takleh nak ambik air.
Tgk jugak aku kat rumah ni.
Pastu aku nak bagi aku.
So aku dah takde masalah pulak.
Balik aku pun
2: penat bak hang, macam ni aku takmau kerja dah macam tu.
Tapi semua tu aku ingat cakap, ada cara hidup ni yang kita kena bayar.. pastu kita tak mampu bayar.. kan!!
Takpelah, aku nak cakap, masa yang


In [16]:
# !transformers-cli upload ./gpt2-117M-bahasa-cased

In [17]:
tokenizer = GPT2Tokenizer.from_pretrained('huseinzol05/gpt2-117M-bahasa-cased')

In [18]:
model = GPT2LMHeadModel.from_pretrained('huseinzol05/gpt2-117M-bahasa-cased', pad_token_id=tokenizer.eos_token_id)

In [19]:
input_ids = tokenizer.encode('penat bak hang, macam ni aku takmau kerja dah', return_tensors='pt')
input_ids

tensor([[ 2809,   266,  1681, 12167,    15,  1673,   656,   476,   558, 48854,
          1470,   826]])

In [ ]:
sample_outputs = model.generate(
    input_ids,
    do_sample=True, 
    max_length=50, 
    top_k=50, 
    top_p=0.95, 
    num_return_sequences=3
)

print("Output:\n" + 100 * '-')
for i, sample_output in enumerate(sample_outputs):
    print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))